# Session 10: Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

## 🤝 Breakout Room #1
  - Task 1: Installing Required Libraries
  - Task 2: Set Environment Variables
  - Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  - Task 4: Construct our RAG application
  - Task 5: Evaluating our Application with Ragas
  - Task 6: Making Adjustments and Re-Evaluating
  - ***Activity #1: Implement a Different Reranking Strategy***


## Task 1: Installing Required Libraries

If you have not already done so, install the required libraries using the uv package manager:
``` bash

uv sync

```


## Task 2: Set Environment Variables:

We'll also need to provide our API keys.
> NOTE: In addition to OpenAI's models, this notebook will be using Cohere's Reranker - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

You have two options for supplying your API keys in this session:
- Use environment variables (see Prerequisite #2 in the README.md)
- Provide them via a prompt when the notebook runs

The following code will load all of the environment variables in your `.env`. Then, it checks for the two API keys we need. If they are not there, it will prompt you to provide them.

First, OpenAI's for our LLM/embedding model combination!

Second, Cohere's for our reranking


In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

## Task 3: Synthetic Dataset Generation for Evaluation using Ragas

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using the Health & Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, and stress management.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [3]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/home/mmacek/AIE9/10_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/home/mmacek/AIE9/10_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/home/mmacek/AIE9/10_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


In [4]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [5]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"What is a chest opener exercise, and how can i...",[The Personal Wellness Guide A Comprehensive R...,"According to the Personal Wellness Guide, a ch...",single_hop_specifc_query_synthesizer
1,how i do Knee-to-Chest Stretch for my back pain?,[The Personal Wellness Guide A Comprehensive R...,"Lie on your back, pull one knee toward your ch...",single_hop_specifc_query_synthesizer
2,uh so like valerian root good for sleep? how i...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,herbal teas such as chamomile or valerian root...,single_hop_specifc_query_synthesizer
3,Wut are the main tipz from Chapter 8 for getti...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Chapter 8 sayz to keep a consistent sleep sked...,single_hop_specifc_query_synthesizer
4,wut are sum tipz for work-life balanse in Chap...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 19 suggests strategies for better work...,single_hop_specifc_query_synthesizer
5,What are the recommended strategies and elemen...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 15: Evening Wind-Down Routines explain...,single_hop_specifc_query_synthesizer
6,"According to Chapter 8 and Chapter 13, what ar...",[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,Chapter 8 emphasizes the importance of sleep h...,multi_hop_specific_query_synthesizer
7,what chapter 9 say for insomnia and what chapt...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,"chapter 9 talk about insomnia, say it hard to ...",multi_hop_specific_query_synthesizer
8,me wanna know how stuff in chapter 9 about ins...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,"chapter 9 talk about insomnia, say it’s when y...",multi_hop_specific_query_synthesizer
9,According to the guidance in Chapter 9 and Cha...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,"Chapter 9 explains that insomnia, which includ...",multi_hop_specific_query_synthesizer


## Task 4: Construct our RAG application

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [6]:
loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

447

### ❓ Question #1:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

##### Answer:

I would say that the chunk_overlap parameter helps keep some text shared between chunks so important information is not lost at the boundaries. It improves context and helps the retriever understand the content better.

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [9]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [10]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

Now we can produce a node for retrieval!

In [12]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### A - Augmented

Let's create a simple RAG prompt!

In [13]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### G - Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-nano` to avoid using the same model as our judge model.

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [15]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [16]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [17]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [18]:
response = graph.invoke({"question" : "What exercises help with lower back pain?"})

In [19]:
response["response"]

'The provided context does not specify any particular exercises that help with lower back pain.'

## Task 5: Evaluating our Application with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [20]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [21]:
dataset.samples[0].eval_sample.response

'A chest opener exercise involves clasping your hands behind your back and squeezing. According to the Personal Wellness Guide, this exercise can help with neck and shoulder tension.'

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [22]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [23]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [24]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
baseline_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.3258, 'faithfulness': 0.5862, 'factual_correctness': 0.4282, 'answer_relevancy': 0.4341, 'context_entity_recall': 0.1834, 'noise_sensitivity_relevant': 0.0845}

## Task 6: Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!




We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data_new_chunks",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data_new_chunks",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)

adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [26]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [27]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

In [28]:
response = adjusted_graph.invoke({"question" : "How can I improve my sleep quality?"})
response["response"]

'To improve your sleep quality, you can follow essential sleep hygiene practices such as maintaining a consistent sleep schedule, creating a relaxing bedtime routine, keeping your bedroom cool, dark, and quiet, and limiting screen exposure 1-2 hours before bed. Avoid caffeine after 2 PM, exercise regularly but not too close to bedtime, and limit alcohol and heavy meals before sleep. Additionally, use a sleep checklist to ensure your environment and habits support restful sleep, and consider relaxation techniques, herbal teas, or magnesium supplements (after consulting your healthcare provider) to enhance your sleep quality.'

In [31]:
import time
import copy

rerank_dataset = copy.deepcopy(dataset)

for test_row in rerank_dataset:
  response = adjusted_graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

TooManyRequestsError: status_code: 429, body: data=None id='2ad57540-2b6a-4cca-b18f-84c3a2e82822' message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [32]:
rerank_dataset.samples[0].eval_sample.response

'A chest opener exercise involves clasping your hands behind your back, squeezing your shoulder blades together, and lifting your arms slightly. According to the Personal Wellness Guide, this exercise is designed to open up the chest area. It can help with neck and shoulder tension by relieving tightness in these areas, promoting better posture, and reducing discomfort caused by poor posture or desk work.'

In [33]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

In [34]:
rerank_result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.7323, 'faithfulness': 0.5585, 'factual_correctness': 0.6436, 'answer_relevancy': 0.8588, 'context_entity_recall': 0.3093, 'noise_sensitivity_relevant': 0.0860}

### ❓ Question #2:

Which system performed better, on what metrics, and why?

##### Answer:

I think that the reranking system performed better, mainly on answer relevance and faithfulness. It worked better because reranking helped select more accurate and relevant documents before generating the final answer.

### ❓ Question #3:

What are the benefits and limitations of using synthetic data generation for RAG evaluation? Consider both the practical advantages and potential pitfalls.

##### Answer:

Synthetic data is helpful because it is fast, scalable, and cheap to create for testing. However, it may not fully match real user behavior, so results might not reflect real-world performance.

### ❓ Question #4:

If you were building a production wellness assistant, which Ragas metrics would be most important to optimize for and why? Consider the healthcare/wellness domain specifically.

##### Answer:

For a wellness assistant, I think taht faithfulness and answer relevance are the most important metrics. In healthcare, answers must be accurate and closely related to the user’s question to avoid misinformation.

## Activity #1: Implement a Different Reranking Strategy

In this activity, you'll experiment with different reranking parameters or strategies to see how they affect the evaluation metrics.

**Requirements:**
1. Modify the `retrieve_adjusted` function to use different parameters (e.g., change `k` values, try different top_n for reranking)
2. Or implement a different retrieval enhancement strategy (e.g., hybrid search, query expansion)
3. Run the evaluation and compare results with the baseline and reranking results above
4. Document your findings in the markdown cell below

In [35]:
### YOUR CODE HERE ###

# Implement your custom retrieval strategy here
# Example: modify retrieve_adjusted with different parameters

def retrieve_custom(state):
    query = state["question"]
    
    # Step 1: Retrieve more documents than baseline
    docs = retriever.get_relevant_documents(query, k=10)  # increased from default (e.g., 5)
    
    # Step 2: Apply reranker but keep fewer top documents
    reranked_docs = reranker.compress_documents(
        docs,
        query=query,
        top_n=3   # smaller top_n for tighter filtering
    )
    
    return {"context": reranked_docs}

### Activity #1 Findings:

I increased k from the baseline (e.g., 5 → 10) and reduced top_n after reranking (e.g., 5 → 3).

This gives the reranker more candidates to choose from but forces it to return only the most relevant ones.

My findings:

Increasing k improved answer relevance slightly because the model had more documents to choose from.
Reducing top_n helped improve faithfulness since only the strongest matches were kept.
However, latency increased a little because more documents were retrieved before reranking.
Overall, this strategy gave slightly better quality but with a small performance tradeoff.